# We're Gonna Need a Bigger Bot

Let's map all the bits and pieces to a meatier example, to see how hello-ltr's abstractions make it easier to play with LTR via ipynbs

Genome-tags is a crowdsourced movie tagging resource project. Each movie is assigned from 0-1 how close a movie matches a tag. Luckily the tags look remarkably like search queries (ie `star trek` or `berlin`). We derrived judgments from the genome-tags data, and use them to experiment with search.

BUT

While some tags are straight forward (`Star Trek`) others are much tougher (`boxing` or `french art movie`) where its unlikely any text has a match. 

We're not going to solve those problems here, but we give this to you as a sandbox to apply your skills after the class to see how close you can get to approximating the genome tags data. 


### Clients 

While syntaxes differ, the LTR process is nearly identical between Solr and Elastic. So you can repeat a lot of the labs with Solr (some have been already translated)

But we'll stick with Elastic

In [2]:
from ltr.client import ElasticClient
client = ElasticClient()

### Reindex if you need to

In [22]:
from ltr.index import rebuild_tmdb

def add_collection_name(src_movie, base_doc):
    if 'belongs_to_collection' in src_movie and src_movie['belongs_to_collection'] is not None:
        if 'name' in src_movie['belongs_to_collection']:
            base_doc['collection_name'] = src_movie['belongs_to_collection']['name']
    if 'production_countries' in src_movie:
        prodCountries = []
        for prod_country in src_movie['production_countries']:
            prodCountries.append(prod_country['name'])
        base_doc['production_countries'] = prodCountries
            
    if 'cast' in src_movie:
        chars = []
        for people in src_movie['cast']:
            chars.append(people['character'])
        base_doc['characters'] = chars

    return base_doc

from ltr.index import rebuild_tmdb
rebuild_tmdb(client, enrich=add_collection_name)

###  Feature Sets in ipynb

You played with creating a feature set in the last lab, see the same process repeated here.

Learning to rank requires creating feature set. Each feature has a name like `title_bm25` and as part of a list an ordinal `title_bm25` is the 0th item. Confusingly, Ranklib uses 1-based feature numbering, so feature 0 in this list corresponds to feature 1 in Ranklib training file, that we'll see soon.

Notice also:

- Each feature is a templated query with `{{keywords}}` parameter, that is passed at query time
- We've added a `validation` block, which will run these queries with the specified parameters and index and return any query errors

In [28]:
config = {
    "featureset": {
        "features": [
            {
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    } 
                }
            },
            {
                "name": "title_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title_phrase": "{{keywords}}"
                    } 
                }
            },
            {
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                        "match": {
                            "overview": {
                                "query": "{{keywords}}",
                            }
                        }

                }
            },
            {
                "name": "tagline_bm25",
                "params": ["keywords"],
                "template": {
                        "match": {
                            "tagline": {
                                "query": "{{keywords}}",
                            }
                        }

                }
            },
            {
                "name": "text_std_bm25",
                "params": ["keywords"],
                "template": {
                        "match": {
                            "text_std": {
                                "query": "{{keywords}}",
                            }
                        }

                }
            },
               {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            },
              {
                "name": "coll_name_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"collection_name": 
                                {"query": "{{keywords}}"}}
                }
            },
            {
                "name": "text_all_phrase_bm25",
                "params": ["keywords"],
                "template": {
                    "match_phrase": {"text_all": "{{keywords}}"}
                }
            },
            {
                "name": "genres_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {"genres": "{{keywords}}"}
                }
            },
            {
                "name": "production_countries_match",
                "params": ["keywords"],
                "template": {
                 "constant_score": {
                    "filter": {
                        "match": {"production_countries": "{{keywords}}"}
                    },
                    "boost": 1.0
                } }
            },
                {
                "name": "characters_match",
                "params": ["keywords"],
                "template": {
                 "constant_score": {
                    "filter": {
                        "match": {"characters": "{{keywords}}"}
                    },
                    "boost": 1.0
                } }
            },
               {
                "name": "text_all_phrase",
                "params": ["keywords"],
                "template": {
                     "constant_score": {
                         "filter": {
                            "match_phrase": {"text_all": "{{keywords}}"}
                         },
                         "boost": 1.0
                    }
                }
            },
        ]
    },
    "validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo cat"
              }
    
           }
}


from ltr import setup
setup(client, config=config, index='tmdb', featureset='genome')

### Logging Queries

Logging is one of the more complex operations from an engineering perspective. 

The same query you ran manually when reviewing the slides is rerun here for every query in the source judgment list `judgmentInFile` with some batching when needed.

In [29]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client,
                                        judgmentInFile='data/genome_judgments.txt', 
                                        trainingOutFile='data/genome_judgments_train.txt', 
                                        featureSet='genome')

### Training

Here's where we train the model, under the hood this executes Ranklib just as you ran during the training exercises.

Notice here we're optimizing for NDCG@10


In [26]:
from ltr.train import train, kcv
trainLog = kcv(client,
               trainingInFile='data/genome_judgments_train.txt',
               metric2t='DCG@10',
               featureSet='genome',
               index='tmdb',
               ranker=8,
               kcv=2,
               frate=1.0,
               srate=1.0,
               modelName='genome')


In [27]:
trainLog.kcvTestAvg

Now that training is done, we can output some statistics about the model, including the training metrics. In future units we'll get more into what this looks like.

Notice the training NDCG isn't that great. When originally run, it was only 0.5885. So pretty far off of the genome data. One challenge of Learning to Rank (and Relevance in general) is trying to figure out the features that can close the gap. 

In [ ]:
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("trainLog Metric %s" % trainLog.metric())

### Search with our model

Here we're going to search using the `genome` model. You can see the LTR query being output (sent to Elasticsearch). You're encouraged to run that directly against Elasticsearch if you like.

Please note, this isn't rescoring. And that's fine for our purposes of directly evaluating the model, in real life you really should run a rescore query.

In [ ]:
from ltr import search
search(client, "batman", modelName='genome')